In [2]:
import pyspark
from delta.pip_utils import configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')  \
    .config("spark.hadoop.fs.s3a.endpoint", "localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

my_packages = ["org.apache.hadoop:hadoop-aws:3.3.1"]
spark = configure_spark_with_delta_pip(builder, extra_packages=my_packages).getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [2]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

23/07/12 17:48:06 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
spark.read.format("delta").load("s3a://data/mobile-reward-share").createOrReplaceTempView("mobile_reward_share")


23/07/12 17:48:08 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/07/12 17:48:08 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
23/07/12 17:48:08 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
23/07/12 17:48:08 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
23/07/12 17:48:08 WARN BasicProfileConfig

In [5]:
spark.sql("""\
SELECT *
FROM mobile_reward_share
limit 5
""").show(truncate)

root
 |-- start_period: decimal(23,0) (nullable = true)
 |-- end_period: decimal(23,0) (nullable = true)
 |-- gateway_reward: struct (nullable = true)
 |    |-- hotspot_key: binary (nullable = true)
 |    |-- beacon_amount: decimal(23,0) (nullable = true)
 |    |-- witness_amount: decimal(23,0) (nullable = true)
 |    |-- dc_transfer_amount: decimal(23,0) (nullable = true)
 |-- operational_reward: struct (nullable = true)
 |    |-- amount: decimal(23,0) (nullable = true)
 |-- date: date (nullable = false)
 |-- file: string (nullable = true)

